In [1]:
!pip install torch torchvision
!pip install torch-geometric
!pip install networkx scikit-learn numpy pandas tqdm

In [2]:
import os
import random
from typing import List, Tuple
import numpy as np
import pandas as pd
import networkx as nx
import pickle

from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.utils import from_networkx
from torch_geometric.nn import SAGEConv

random.seed(42)
np.random.seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_source="SemanticGraph_delta_1_cutoff_0_minedge_1.pkl"
save_predictions_path = "predictions.csv"

C:\Users\kuzzm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch_geometric\__init__.py:4: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: [WinError 127] Не найдена указанная процедура
  import torch_geometric.typing
C:\Users\kuzzm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch_geometric\__init__.py:4: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: [WinError 127] Не найдена указанная процедура
  import torch_geometric.typing
C:\Users\kuzzm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch_geometric\__init__.py:4: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: [WinError 127] Не найдена указанная про

Help functions:

In [3]:
def structural_features(G: nx.Graph, u: int, v: int, max_shortest=6):
    """
    Computes set of structural graph features for a node pair (u, v):
        1. Common Neighbors: Number of nodes connected to both u and v
        2. Jaccard Coefficient: Normalized measure of neighborhood overlap
        3. Adamic-Adar Index: Weighted version of common neighbors (favors rare neighbors)
        4. Node Degrees: Connectivity information for both nodes
        5. Shortest Path Length: Distance between nodes (capped for efficiency)
    """
    # Common Neighbors
    try:
        cn = len(list(nx.common_neighbors(G, u, v)))
    except Exception:
        cn = 0
    
    # Jaccard Coefficient
    j = 0.0
    try:
        jc = next(nx.jaccard_coefficient(G, [(u,v)]))[2]
        j = jc if jc is not None else 0.0
    except Exception:
        j = 0.0
    
    # Adamic-Adar Index
    aa = 0.0
    try:
        aa = next(nx.adamic_adar_index(G, [(u,v)]))[2] or 0.0
    except Exception:
        aa = 0.0
    
    # Node Degrees: Number of edges incident to each node
    # Provides information about node importance/centrality in the network
    du = G.degree(u)
    dv = G.degree(v)
    
    # Shortest Path Length: Minimum number of edges to traverse from u to v
    # Capped at max_shortest to avoid expensive computations for disconnected nodes
    try:
        spl = nx.shortest_path_length(G, source=u, target=v)
        if spl > max_shortest:
            spl = max_shortest
    except Exception:
        # Nodes are disconnected (no path exists)
        spl = max_shortest
    
    return [cn, j, aa, du, dv, spl]

def build_pair_features(pairs: List[Tuple[int,int]], emb: np.ndarray, G: nx.Graph):
    """
    Constructs a comprehensive feature matrix for node pairs by combining learned embeddings
    with structural graph features.
    """
    feats = []
    for (u, v) in pairs:
        # Extract learned embeddings for both nodes
        zu = emb[u]
        zv = emb[v]
        
        # Vector-based features from embeddings:
        # - Concatenation of both embeddings (preserves individual node information)
        # - Element-wise product (captures interaction patterns)
        # - Absolute difference (captures similarity/dissimilarity)
        vec = np.concatenate([zu, zv, zu * zv, np.abs(zu - zv)], axis=0)
        
        # Structural features from graph topology
        sf = structural_features(G, u, v)
        
        # Combine embedding-based and structural features into single feature vector
        feat = np.concatenate([vec, np.array(sf, dtype=float)], axis=0)
        feats.append(feat)
    
    return np.vstack(feats)

def sample_negatives(G: nx.Graph, num_samples: int, forbid_set:set):
    """
    Samples random negative (non-edge) pairs from the graph for training.
    """
    negs = set()
    n = G.number_of_nodes()
    
    while len(negs) < num_samples:
        u = random.randrange(n)
        v = random.randrange(n)
        
        # Skip self-loops
        if u == v:
            continue
        
        # Normalize pair ordering (u < v) for consistent representation
        key = (min(u, v), max(u, v))
        
        # Skip if this pair is forbidden (exists in training or future edges)
        if key in forbid_set:
            continue
        
        negs.add(key)
    
    return list(negs)

def precision_at_k(y_true: np.ndarray, y_score: np.ndarray, K=100):
    """
    Computes Precision@K metric for link prediction evaluation.
    """
    idx = np.argsort(-y_score)[:K]
    return y_true[idx].sum() / float(K)

GraphSAGE encoder and pair-MLP:

In [4]:
class GraphSAGEEncoder(nn.Module):
    """
    GraphSAGE encoder for learning node embeddings.
    
    The encoder uses GraphSAGE convolutional layers which:
    - Sample a fixed-size neighborhood for each node
    - Aggregate neighbor features
    - Combine aggregated neighbor features with the node's own features
    - Apply non-linear activation (ReLU) to learn complex patterns
    """

    def __init__(self, in_channels, hidden_channels=128, num_layers=2):
        super().__init__()
        self.convs = nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 1):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
    
    def forward(self, x, edge_index):
        for conv in self.convs:
            x = conv(x, edge_index)
            x = F.relu(x)
        return x

class PairMLP(nn.Module):
    """
    Multi-Layer Perceptron (MLP) classifier for link prediction on node pairs.
    
    Architecture:
        Input -> FC1 (hidden1) -> ReLU -> FC2 (hidden2) -> ReLU -> Output (1) -> Sigmoid
    
    Attributes:
        fc1 (nn.Linear): First fully connected layer mapping input to first hidden dimension
        fc2 (nn.Linear): Second fully connected layer mapping first to second hidden dimension
        out (nn.Linear): Output layer mapping second hidden dimension to single logit
    
    Args:
        in_dim (int): Dimensionality of input feature vectors (4*embedding_dim + 6)
        hidden1 (int, optional): Size of first hidden layer. Defaults to 256.
        hidden2 (int, optional): Size of second hidden layer. Defaults to 64.
    """
    def __init__(self, in_dim, hidden1=256, hidden2=64):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, 1)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return torch.sigmoid(self.out(x)).squeeze()

Load dataset:

In [5]:
with open(data_source, "rb" ) as pkl_file:
    full_dynamic_graph_sparse, unconnected_vertex_pairs, unconnected_vertex_pairs_solution, year_start, years_delta, vertex_degree_cutoff, min_edges = pickle.load(pkl_file)

Create graph and node features:

In [6]:
G_train = nx.Graph()

# infer maximum node id from the edges
max_id = int(max(full_dynamic_graph_sparse[:, 0].max(),
                 full_dynamic_graph_sparse[:, 1].max()))

G_train.add_nodes_from(range(max_id + 1))
G_train.add_edges_from(zip(full_dynamic_graph_sparse[:, 0],
                           full_dynamic_graph_sparse[:, 1]))

n_nodes = max_id + 1

deg = np.array([G_train.degree(i) for i in range(n_nodes)], dtype=float).reshape(-1, 1)

# Normalize degrees
deg = deg / (deg.max() + 1e-9)

# Random node features
rand_feat = np.random.randn(n_nodes, 16).astype(np.float32)

# Final node feature matrix 
node_features = np.concatenate([deg.astype(np.float32), rand_feat], axis=1)  # shape (n_nodes, 17)

Train GraphSAGE encoder to learn node embeddings from graph structure:

In [7]:
num_epochs = 30

pyg_data = from_networkx(G_train)
pyg_data.x = torch.tensor(node_features, dtype=torch.float32)
pyg_data = pyg_data.to(device)

encoder = GraphSAGEEncoder(in_channels=pyg_data.x.size(1), hidden_channels=128, num_layers=2).to(device)
optimizer = torch.optim.Adam(encoder.parameters(), lr=0.01, weight_decay=1e-4)
encoder.train()

edge_index = pyg_data.edge_index

# learn embeddings using link prediction as self-supervised task
for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    # compute node embeddings
    emb = encoder(pyg_data.x, pyg_data.edge_index)
    
    # extract source and destination nodes for all edges
    src = edge_index[0]
    dst = edge_index[1]
    
    # similarity scores for positive pairs using dot product
    pos_scores = (emb[src] * emb[dst]).sum(dim=1)
    
    # Positive loss
    pos_loss = -F.logsigmoid(pos_scores).mean()
    
    # randomly sampled non-edges
    n_pos = src.size(0)
    
    # Sample random negative pairs: keep source nodes, randomize destinations
    neg_u = src[torch.randint(0, n_pos, (n_pos,)).to(device)]
    neg_v = torch.randint(0, emb.size(0), (n_pos,)).to(device)
    
    # Compute similarity scores for negative pairs
    neg_scores = (emb[neg_u] * emb[neg_v]).sum(dim=1)
    
    # Negative loss
    neg_loss = -F.logsigmoid(-neg_scores).mean()
    
    # Total loss
    loss = pos_loss + neg_loss
    
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {epoch} loss {loss.item():.4f}")

# final node embeddings
encoder.eval()
with torch.no_grad():
    node_emb = encoder(pyg_data.x, pyg_data.edge_index).cpu().numpy()

RuntimeError: [enforce fail at alloc_cpu.cpp:115] data. DefaultCPUAllocator: not enough memory: you tried to allocate 11908202496 bytes.

Prepare X and y:

In [ ]:
# candidate pairs and labels from dataset
pairs_all = [tuple(p) for p in unconnected_vertex_pairs]
pairs_all = [(min(u, v), max(u, v)) for (u, v) in pairs_all]

# Labels: 1 if the pair gains >= min_edges edges between year_start and year_start+years_delta
y = np.array(unconnected_vertex_pairs_solution, dtype=int)

print("Pairs total:", len(pairs_all), "positives:", int(y.sum()), "negatives:", int((1 - y).sum()))

# Build feature matrix for all candidate pairs using G_train and learned embeddings
X = build_pair_features(pairs_all, node_emb, G_train)
print("Feature matrix shape:", X.shape)

Train MLP classifier to predict link existence from pair features:

In [ ]:
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.25, random_state=42, stratify=y)

X_train_t = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_t = torch.tensor(y_train, dtype=torch.float32).to(device)
X_val_t = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_t = torch.tensor(y_val, dtype=torch.float32).to(device)

model = PairMLP(X_train.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
batch_size = 128
num_epochs = 40

for epoch in range(num_epochs):
    # Shuffle training data each epoch for better generalization
    perm = torch.randperm(X_train_t.size(0))
    losses = []
    model.train()
    
    for i in range(0, X_train_t.size(0), batch_size):
        idx = perm[i:i+batch_size]
        xb = X_train_t[idx]
        yb = y_train_t[idx]
        
        optimizer.zero_grad()
        out = model(xb)
     
        loss = F.binary_cross_entropy(out, yb)
        
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    
    # evaluation on validation set 
    model.eval()
    with torch.no_grad():
        preds = model(X_val_t).cpu().numpy()
        auc = roc_auc_score(y_val, preds)
        ap = average_precision_score(y_val, preds)
    model.train()
    print(f"Epoch {epoch} train_loss={np.mean(losses):.4f} val_auc={auc:.4f} val_ap={ap:.4f}")

Final evaluation on the complete dataset (training + validation)

In [ ]:
model.eval()
with torch.no_grad():
    preds_all = model(torch.tensor(X_scaled, dtype=torch.float32).to(device)).cpu().numpy()

auc_all = roc_auc_score(y, preds_all)
ap_all = average_precision_score(y, preds_all)
prec100 = precision_at_k(y, preds_all, K=min(100, len(y)))
print("Final metrics: AUC", auc_all, "AP", ap_all, "Precision@100", prec100)

# Save predictions to CSV file
out_df = pd.DataFrame({
    "u": [p[0] for p in pairs_all],
    "v": [p[1] for p in pairs_all],
    "label": list(y.astype(int)), # 1=edge, 0=non-edge
    "score": list(preds_all)
})
os.makedirs(os.path.dirname(save_predictions_path), exist_ok=True)
out_df.to_csv(save_predictions_path, index=False)
print("Saved predictions to", save_predictions_path)